In [16]:
# !pip install "google-cloud-aiplatform>=1.38" vertexai
#TODO: gcloud auth application-default login to authenticate

## Preprocess input

In [17]:
predictions_root_dir = '/home/watson_chua/efs/axolotl/data/predictions/hansard/'
llama_predictions_path = predictions_root_dir + "llama3_8b_lora.jsonl"
gemma_predictions_path = predictions_root_dir + "gemma2_9b_lora.jsonl"
gpt4_predictions_path, out_subdir = "/home/watson_chua/efs/axolotl/data/input_data/data/gpt4_answers_by_hy_doc.jsonl", 'gpt4_normal'
# gpt4_predictions_path, out_subdir = "/home/watson_chua/efs/axolotl/data/input_data/data/gpt4_answers_by_hy_doc_concise.jsonl", 'gpt4_concise'


In [18]:
import json
with open(llama_predictions_path, 'r') as f:
    llama_lines = f.readlines()

with open(gemma_predictions_path, 'r') as f:
    gemma_lines = f.readlines()

with open(gpt4_predictions_path, 'r') as f:
    gpt4_lines = f.readlines()

llama_lines = [json.loads(l) for l in llama_lines]
gemma_lines = [json.loads(l) for l in gemma_lines]
gpt4_lines = [json.loads(l) for l in gpt4_lines]

In [19]:
len(llama_lines), len(gemma_lines), len(gpt4_lines)

(304, 304, 304)

In [20]:
llama_lines[0]

{'generation': 'As at end-January 2024, there were 101,464 persons who did not vote in the last Presidential election but have since applied to restore their names to the Registers of Electors. All such applications have been approved after being verified by the Elections Department.',
 'input': "<|start_header_id|>system<|end_header_id|>You are a public servant. Your task is to reply to a parliamentary question given a list of supporting points.<|eot_id|><|start_header_id|>user<|end_header_id|>\nQuestion:Mr Leong Mun Wai asked the Prime Minister (a) since the conclusion of the 2023 Presidential Election, how many non-voters have applied to restore their names to the Registers of Electors; and (b) how many of these applications are successful.\n\nSupporting points: **Title: Applications to Restore Names to Registers of Electors**\n\n**Executive Summary:**\n\nAs of the end of January 2024, a total of 101,464 non-voters from the 2023 Presidential Election have successfully applied to res

In [21]:
gpt4_lines[0]['question']

'Mr Leong Mun Wai asked the Prime Minister (a) since the conclusion of the 2023 Presidential Election, how many non-voters have applied to restore their names to the Registers of Electors; and (b) how many of these applications are successful.'

In [22]:
llama_lines[0]['input'].split("<|end_header_id|>\nQuestion:", maxsplit=1)[-1].split("\n\nSupporting points:")[0]

'Mr Leong Mun Wai asked the Prime Minister (a) since the conclusion of the 2023 Presidential Election, how many non-voters have applied to restore their names to the Registers of Electors; and (b) how many of these applications are successful.'

In [23]:
llama_questions = [q['input'].split("<|end_header_id|>\nQuestion:", maxsplit=1)[-1].split("\n\nSupporting points:")[0] for q in llama_lines]

In [24]:
for i,l in enumerate(gpt4_lines):
    if l['question'] not in llama_questions:
        print(i, l['question'])

In [25]:
import pandas as pd 
df_gpt4 = pd.DataFrame(gpt4_lines)

In [26]:
df_duplicated = df_gpt4[df_gpt4['question'].duplicated()]

In [27]:
df_gpt4_deduplicated = df_gpt4.drop_duplicates('question')

In [28]:
df_gpt4_deduplicated.head()

,status,reference_answer,summary_points,points,title,subtitle,question_speaker,answer_speaker,question,answer,filename,hypothetical_document,date,gpt4_answer_by_hy_doc
0,answered,,"[101,464 non-voters from the 2023 Presidential...","101,464 non-voters from the 2023 Presidential ...",Applications to Restore Names to Registers of ...,None,Mr Leong Mun Wai,Mr Chan Chun Sing (for the Prime Minister),Mr Leong Mun Wai asked the Prime Minister (a) ...,Mr Chan Chun Sing (for the Prime Minister): As...,sitting_2024-02-29,**Title: Applications to Restore Names to Regi...,2024-02-29 00:00:00,"As of the end of January 2024, a total of 101,..."
1,answered,,"[From 2021 to 2023, over 23,300 companies obta...","From 2021 to 2023, over 23,300 companies obtai...",Applications under Enterprise Financing Scheme,None,Mr Liang Eng Hwa,Mr Gan Kim Yong,Mr Liang Eng Hwa asked the Minister for Trade ...,"Mr Gan Kim Yong: From 2021 to 2023, over 23,30...",sitting_2024-02-29,**Title: Applications under Enterprise Financi...,2024-02-29 00:00:00,"From 2021 to 2023, over 23,300 companies have ..."
2,answered,,"[About 36,400 companies have received support ...","About 36,400 companies have received support u...",Applications for Productivity Solutions Grant ...,None,Mr Liang Eng Hwa,Mr Gan Kim Yong,Mr Liang Eng Hwa asked the Minister for Trade ...,"Mr Gan Kim Yong: In the last three years, abou...",sitting_2024-02-29,### Applications for Productivity Solutions Gr...,2024-02-29 00:00:00,"Over the last three years, approximately 36,40..."
3,answered,,[Schools have autonomy to decide the end times...,Schools have autonomy to decide the end times ...,Time Guidelines for Schools to Cease Co-curric...,None,Mr Leong Mun Wai,Mr Chan Chun Sing,Mr Leong Mun Wai asked the Minister for Educat...,Mr Chan Chun Sing: Schools have the autonomy t...,sitting_2024-02-29,**Title: Time Guidelines for Schools to Cease ...,2024-02-29 00:00:00,"Yes, there are overarching guidelines for scho..."
4,answered,,[Rail operators have improved track design and...,Rail operators have improved track design and ...,Mitigation Measures to Reduce Noise from Railw...,None,Mr Leong Mun Wai,Mr Chee Hong Tat,Mr Leong Mun Wai asked the Minister for Transp...,"Over the years, rail operators have improved t...",sitting_2024-02-29,**Report: Mitigation Measures to Reduce Noise ...,2024-02-29 00:00:00,"**Reply:**\n\nMr Speaker,\n\nTo address the no..."


In [29]:
df_all_predictions = df_gpt4_deduplicated.copy()

In [30]:
df_all_predictions['llama3_answer'] = [l['generation']  for l in llama_lines]
df_all_predictions['gemma2_answer'] = [l['generation']  for l in gemma_lines]

In [31]:
output_root_dir = '/home/watson_chua/efs/axolotl/data/evaluations/hansard/'

In [33]:
df_all_predictions.to_csv(output_root_dir + out_subdir + '/consolidated_predictions.csv', index=False)

## LLM Evaluation

In [42]:
output_root_dir = '/home/watson_chua/efs/axolotl/data/evaluations/hansard/'
subdir = 'gpt4_normal'
# subdir = 'gpt4_concise'



In [44]:
import pandas as pd
df_all_predictions = pd.read_csv(output_root_dir + subdir + '/consolidated_predictions.csv')

In [36]:
import vertexai
from vertexai.generative_models import GenerativeModel

# TODO(developer): Update and un-comment below line
project_id = "gvt0031-gcp-152-govtext-ds"

vertexai.init(project=project_id, location="asia-southeast1")

model = GenerativeModel(model_name="gemini-1.5-flash-001")

In [45]:
prompt_three_way_template = """Given the following question, context, and three different answers a), b) and c), assess the answer based on the following criteria: 
 1) factual correctness according to the context
 2) similarity to model answer
 3) conciseness
 
Respond in JSON whether answer a), b), or c) is the better answer and state your reason using the following schema:

{{"winner": a, b, or c,
"reason": reason it is the better answer}}

Here is the information,

{question}

Context: {context}

Model Answer: {ground_truth}

Answer A: {answer_a}

Answer B: {answer_b}

Answer C: {answer_c}



Respond with only the JSON reply and nothing else. Use double quotes for the json schema and DO NOT use any double quotes in the values.
"""


prompt_two_way_template = """Given the following question, context, and two different answers a) and b), assess the answer based on the following criteria: 
 1) factual correctness according to the context
 2) similarity to model answer
 3) conciseness
 
Respond in JSON whether answer a) or b) is the better answer and state your reason using the following schema:

{{"winner": a or b,
"reason": reason it is the better answer}}

Here is the information,

{question}

Context: {context}

Model Answer: {ground_truth}

Answer A: {answer_a}

Answer B: {answer_b}

Respond with only the JSON reply and nothing else. Use double quotes for the json schema and DO NOT use any double quotes in the values.
"""

In [38]:
from tqdm.auto import tqdm
three_way_evaluation_results = []
gpt4_llama3_evaluation_results = []
gpt4_gemma2_evaluation_results = []
llama3_gemma2_evaluation_results = []


df_eval = df_all_predictions
for i, (index, row) in tqdm(enumerate(df_eval.iterrows()), total=len(df_eval)):
    gpt_4_answer = row['gpt4_answer_by_hy_doc']
    ground_truth = row['answer']
    question = row['question']
    hypothetical_doc = row['hypothetical_document']
    llama3_answer = row['llama3_answer']
    gemma2_answer = row['gemma2_answer']


    prompt_three_way = prompt_three_way_template.format(context=hypothetical_doc, ground_truth=ground_truth, question=question, answer_a=gpt_4_answer, answer_b=llama3_answer, answer_c=gemma2_answer)
    prompt_gpt4_llama3 = prompt_two_way_template.format(context=hypothetical_doc, ground_truth=ground_truth, question=question, answer_a=gpt_4_answer, answer_b=llama3_answer)
    prompt_gpt4_gemma2 = prompt_two_way_template.format(context=hypothetical_doc, ground_truth=ground_truth, question=question, answer_a=gpt_4_answer, answer_b=gemma2_answer)
    prompt_llama3_gemma2 = prompt_two_way_template.format(context=hypothetical_doc, ground_truth=ground_truth, question=question, answer_a=llama3_answer, answer_b=gemma2_answer)


    try:
        response_three_way = model.generate_content(prompt_three_way).text
        response_gpt4_llama3 = model.generate_content(prompt_gpt4_llama3).text
        response_gpt4_gemma2 = model.generate_content(prompt_gpt4_gemma2).text
        response_llama3_gemma2 = model.generate_content(prompt_llama3_gemma2).text


    except ValueError as e:
        print(e)
        continue
    
    
    three_way_evaluation_results.append({'gemini_flash_evaluation': response_three_way, **row, 'llama3_answer': llama3_answer, 'gemma2_answer': gemma2_answer})
    gpt4_llama3_evaluation_results.append({'gemini_flash_evaluation': response_gpt4_llama3, **row, 'llama3_answer': llama3_answer})
    gpt4_gemma2_evaluation_results.append({'gemini_flash_evaluation': response_gpt4_gemma2, **row, 'gemma2_answer': gemma2_answer})
    llama3_gemma2_evaluation_results.append({'gemini_flash_evaluation': response_llama3_gemma2, **row, 'llama3_answer': llama3_answer, 'gemma2_answer': gemma2_answer})





100%|██████████| 304/304 [23:50<00:00,  4.70s/it]


In [39]:
import json
with open(output_root_dir + subdir + '/gpt4_llama3_gemma2_evaluation.jsonl', 'w') as f:
    for l in three_way_evaluation_results:
        f.write(json.dumps(l) + '\n')  

with open(output_root_dir + subdir + '/gpt4_llama3_evaluation.jsonl', 'w') as f:
    for l in gpt4_llama3_evaluation_results:
        f.write(json.dumps(l) + '\n')  

with open(output_root_dir + subdir + '/gpt4_gemma2_evaluation.jsonl', 'w') as f:
    for l in gpt4_gemma2_evaluation_results:
        f.write(json.dumps(l) + '\n')  

with open(output_root_dir + subdir + '/llama3_gemma2_evaluation.jsonl', 'w') as f:
    for l in llama3_gemma2_evaluation_results:
        f.write(json.dumps(l) + '\n')  

In [40]:
import re
def replace_inner_quotes(text):
    split_result  = re.split("\"\s*reason\s*\"", text, maxsplit=1)
    winner_str = split_result[0] 
    reason_str = split_result[1][split_result[1].index('"')+1: split_result[1].rfind('"')].replace('"','').replace("\\", "")
    return winner_str + "\"reason\": \"" + reason_str.strip() + '"}'

def clean_json_output(output):
    output = output.strip()
    if output.startswith("```json"):
        output = output[7:]
    if output.endswith("```"):
        output = output[:-3]
    cleaned_output = replace_inner_quotes(output.strip())
    # cleaned_output = output.strip()


    try:
        json_data = json.loads(cleaned_output, strict=False)
    except json.JSONDecodeError as e:
        print(f"JSON decoding error: {e}")
        print(cleaned_output)
        return cleaned_output

    def clean_json(data):
        if isinstance(data, dict):
            return {key: clean_json(value) for key, value in data.items()}
        elif isinstance(data, list):
            return [clean_json(item) for item in data]
        elif isinstance(data, str):
            return "" if data.lower() in ["unknown", "na", "null"] else data
        else:
            print(data)
            return data
    
    cleaned_json_data = clean_json(json_data)
    # cleaned_output = json.dumps(cleaned_json_data, ensure_ascii=False)

    return cleaned_json_data

In [41]:
# import re
# def extract_json(text):
#     if text.startswith("```json"):
#         found = re.findall("```json\n*(.*)\n*```", text)
#         try:
#             result = json.loads(found[0])
#         except IndexError as e:
#             print(e, found, text)
#     else:
#         result = json.loads(text)        
#     return result

In [42]:
#Test

for er in three_way_evaluation_results:
    clean_json_output(er['gemini_flash_evaluation'])

In [43]:
three_way_eval_json = [clean_json_output(er['gemini_flash_evaluation']) for er in three_way_evaluation_results]
df_three_way_eval_results = pd.DataFrame(three_way_eval_json)
df_three_way_eval_results.to_csv(output_root_dir + subdir + '/gpt4_llama3_gemma2_evaluation_outcome.csv', index=False)


gpt4_llama3_eval_json = [clean_json_output(er['gemini_flash_evaluation']) for er in gpt4_llama3_evaluation_results]
df_gpt4_llama3_eval_results = pd.DataFrame(gpt4_llama3_eval_json)
df_gpt4_llama3_eval_results.to_csv(output_root_dir + subdir + '/gpt4_llama3_evaluation_outcome.csv', index=False)


gpt4_gemma2_eval_json = [clean_json_output(er['gemini_flash_evaluation']) for er in gpt4_gemma2_evaluation_results]
df_gpt4_gemma2_eval_results = pd.DataFrame(gpt4_gemma2_eval_json)
df_gpt4_gemma2_eval_results.to_csv(output_root_dir + subdir + '/gpt4_gemma2_evaluation_outcome.csv', index=False)

llama3_gemma2_eval_json = [clean_json_output(er['gemini_flash_evaluation']) for er in llama3_gemma2_evaluation_results]
df_llama3_gemma2_eval_results = pd.DataFrame(llama3_gemma2_eval_json)
df_llama3_gemma2_eval_results.to_csv(output_root_dir + subdir + '/llama3_gemma2_evaluation_outcome.csv', index=False)

In [44]:
print(df_three_way_eval_results['winner'].value_counts(),
df_gpt4_llama3_eval_results['winner'].value_counts(),
df_gpt4_gemma2_eval_results['winner'].value_counts(),
df_llama3_gemma2_eval_results['winner'].value_counts())

winner
c    120
a     96
b     88
Name: count, dtype: int64 winner
b    212
a     92
Name: count, dtype: int64 winner
b    221
a     83
Name: count, dtype: int64 winner
b    188
a    116
Name: count, dtype: int64


## ragas faithfulness

In [46]:
faithfulness_prompt_template = """Give the following contexts, separated by ######################### and an answer, do the following:
1. Break down the contexts into a list of statements
2. Break down the answer into a list of statements
3. Match the two list of statements and return the following in JSON with the following keys: "statements_in_both", "statements_in_contexts_only", "statements_in_answer_only"

Return your answer as a JSON string in a JSON parsable format without any other output.

Here are the contexts:
{context}


This is the answer:
{answer}
"""

In [52]:
answer_key = "llama3_answer"

faithfulness_json = []

for index, row in tqdm(df_all_predictions[215:].iterrows(), total=len(df_all_predictions)):
    answer = row[answer_key]
    hy_doc = row['hypothetical_document']
    faithfulness_prompt = faithfulness_prompt_template.format(answer=answer_key, context=hy_doc)
    try:
        response = model.generate_content(faithfulness_prompt)
        faithfulness_json.append(response.text)
    except ValueError as ve:
        print(ve)
        faithfulness_json.append("\{\"error\": " + str(ve) +"\}")
        continue

    

  0%|          | 1/304 [00:00<00:32,  9.31it/s]

Response has no candidates (and thus no text). The response is likely blocked by the safety filters.
Response:
{
  "prompt_feedback": {
    "block_reason": "PROHIBITED_CONTENT"
  },
  "usage_metadata": {
    "prompt_token_count": 1012,
    "total_token_count": 1012
  }
}


 29%|██▉       | 89/304 [10:45<25:58,  7.25s/it]


In [56]:
with open(output_root_dir + '/llama3_faithfulness.jsonl', 'w') as f:
    for fj in faithfulness_json:
        f.write(json.dumps(fj) + '\n')

## Groq models

In [1]:
import os
from getpass import getpass

In [23]:
groq_api_key=getpass()

In [47]:
from groq import Groq

groq_client = Groq(api_key=groq_api_key)

def query_groq(prompt):
    completion = groq_client.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.01,
        max_tokens=500,
        top_p=1,
        stream=False,
        stop=None,
    )

    # groq_response = ""
    # for chunk in completion:
    #     # print(chunk.choices[0].delta.content or "", end="")
    #     # print(chunk.choices[0].delta.content or "", end="")
    #     response_chunk = chunk.choices[0].delta.content
    #     if response_chunk is not None and response_chunk.strip():
    #         groq_response +=  response_chunk + " "
    
    return completion.choices[0].message.content

In [39]:
response = query_groq()


{'name': 'Assistant',
 'address': {'street': '1234 Main St',
  'city': 'Anytown',
  'state': 'CA',
  'postal_code': '12345'}}

In [35]:
json.loads(response)

In [68]:
from anthropic import AnthropicBedrock

client = AnthropicBedrock()

def query_claude3(prompt):
    message = client.messages.create(
        model="anthropic.claude-3-sonnet-20240229-v1:0",
        max_tokens=500,
        messages=[{"role": "user", "content": prompt}]
    )
    return message.content[0].text

In [66]:
from tqdm.auto import tqdm
three_way_evaluation_results_claude = []
gpt4_llama3_evaluation_results_claude = []
gpt4_gemma2_evaluation_results_claude = []
llama3_gemma2_evaluation_results_claude = []

df_eval = df_all_predictions
for i, (index, row) in tqdm(enumerate(df_eval.iterrows()), total=len(df_eval)):
    gpt_4_answer = row['gpt4_answer_by_hy_doc']
    ground_truth = row['answer']
    question = row['question']
    hypothetical_doc = row['hypothetical_document']
    llama3_answer = row['llama3_answer']
    gemma2_answer = row['gemma2_answer']


    prompt_three_way = prompt_three_way_template.format(context=hypothetical_doc, ground_truth=ground_truth, question=question, answer_a=gpt_4_answer, answer_b=llama3_answer, answer_c=gemma2_answer)
    prompt_gpt4_llama3 = prompt_two_way_template.format(context=hypothetical_doc, ground_truth=ground_truth, question=question, answer_a=gpt_4_answer, answer_b=llama3_answer)
    prompt_gpt4_gemma2 = prompt_two_way_template.format(context=hypothetical_doc, ground_truth=ground_truth, question=question, answer_a=gpt_4_answer, answer_b=gemma2_answer)
    prompt_llama3_gemma2 = prompt_two_way_template.format(context=hypothetical_doc, ground_truth=ground_truth, question=question, answer_a=llama3_answer, answer_b=gemma2_answer)


    try:
        response_three_way = query_claude3(prompt_three_way)
        response_gpt4_llama3 = query_claude3(prompt_gpt4_llama3)
        response_gpt4_gemma2 = query_claude3(prompt_gpt4_gemma2)
        response_llama3_gemma2 = query_claude3(prompt_llama3_gemma2)


    except ValueError as e:
        print(e)
        continue
    
    
    three_way_evaluation_results_claude.append({'claude_evaluation': response_three_way, **row, 'llama3_answer': llama3_answer, 'gemma2_answer': gemma2_answer})
    gpt4_llama3_evaluation_results_claude.append({'claude_evaluation': response_gpt4_llama3, **row, 'llama3_answer': llama3_answer})
    gpt4_gemma2_evaluation_results_claude.append({'claude_evaluation': response_gpt4_gemma2, **row, 'gemma2_answer': gemma2_answer})
    llama3_gemma2_evaluation_results_claude.append({'claude_evaluation': response_llama3_gemma2, **row, 'llama3_answer': llama3_answer, 'gemma2_answer': gemma2_answer})



100%|██████████| 3/3 [00:40<00:00, 13.58s/it]


In [67]:
three_way_evaluation_results_claude

[{'claude_evaluation': [TextBlock(text='{\n"winner": "b",\n"reason": "Answer b is the better answer as it is factually correct according to the context, closely resembles the model answer, and is concise and to the point."\n}', type='text')],
  'status': 'answered',
  'reference_answer': nan,
  'summary_points': "['101,464 non-voters from the 2023 Presidential Election applied to restore their names to the Registers of Electors as of end January 2024.', 'All the applications were successful.']",
  'points': '101,464 non-voters from the 2023 Presidential Election applied to restore their names to the Registers of Electors as of end January 2024.\n\nAll the applications were successful.',
  'title': 'Applications to Restore Names to Registers of Electors',
  'subtitle': nan,
  'question_speaker': 'Mr Leong Mun Wai',
  'answer_speaker': 'Mr Chan Chun Sing (for the Prime Minister)',
  'question': 'Mr Leong Mun Wai asked the Prime Minister (a) since the conclusion of the 2023 Presidential E

In [ ]:
import json
# with open(output_root_dir + subdir + '/gpt4_llama3_gemma2_evaluation_mixtral.jsonl', 'w') as f:
#     for l in three_way_evaluation_results:
#         f.write(json.dumps(l) + '\n')  

# with open(output_root_dir + subdir + '/gpt4_llama3_evaluation_mixtral.jsonl', 'w') as f:
#     for l in gpt4_llama3_evaluation_results:
#         f.write(json.dumps(l) + '\n')  

with open(output_root_dir + subdir + '/gpt4_gemma2_evaluation_mixtral.jsonl', 'w') as f:
    for l in gpt4_gemma2_evaluation_results:
        f.write(json.dumps(l) + '\n')  

with open(output_root_dir + subdir + '/llama3_gemma2_evaluation_mixtral.jsonl', 'w') as f:
    for l in llama3_gemma2_evaluation_results:
        f.write(json.dumps(l) + '\n')  

In [58]:
import re
llama3_mixtral_eval = [re.findall("\"winner\": \"(\w)\"", er['mixtral_evaluation']) for er in eval_results]

In [59]:
pd.DataFrame(llama3_mixtral_eval).value_counts()

a    256
b     36
A      8
Name: count, dtype: int64